In [7]:
import os
import json
import numpy as np
import spacy
import onnx
import onnxruntime
import ipywidgets as widgets
from IPython.display import display

In [8]:
# ----------------------
# Load Vocabulary
# ----------------------
def load_vocab(model_name):
    model_path = os.path.join("models", model_name, "vocab-dict.json")
    with open(model_path, 'r') as json_file:
        vocab = json.load(json_file)
        return vocab

# ----------------------
# Load ONNX Model
# ----------------------
def load_model(model_name):
    model_path = os.path.join("models", str(model_name), "model-q.onnx")
    model = onnx.load(model_path)
    onnx.checker.check_model(model)
    ort_session = onnxruntime.InferenceSession(model_path)
    return ort_session

# ----------------------
# Tokenizer
# ----------------------
def load_spacy():
    model_path = os.path.join("data", "en_core_web_sm", "en_core_web_sm-3.8.0")
    if os.path.isdir(model_path):
        return spacy.load(model_path)
    else:
        raise FileNotFoundError(f"SpaCy model not found at {model_path}. Please ensure it is correctly placed.")
spacy_en = load_spacy()

def tokenizer(text):
    return [
        tok.text.lower() 
        for tok in spacy_en.tokenizer(text) 
        if not tok.is_punct and not tok.is_space
    ]

# ----------------------
# Predict Function
# ----------------------
def softmax(x, axis=None):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

def predict_sentiment(ort_session, vocab, sequence, max_length=100):
    tokens = sequence.strip().split()
    token_indices = [vocab.get(token, 0) for token in tokens]
    
    if len(token_indices) < max_length:
        token_indices += [0] * (max_length - len(token_indices))
    else:
        token_indices = token_indices[:max_length]
    
    input_array = np.array(token_indices, dtype=np.int64).reshape(1, -1)
    ort_inputs = {'input': input_array}
    ort_outs = ort_session.run(None, ort_inputs)
    logits = ort_outs[0]
    probabilities = softmax(logits, axis=1)
    confidence = np.max(probabilities, axis=1)[0]
    label = np.argmax(probabilities, axis=1)[0]
    sentiment = 'positive' if label == 1 else 'negative'
    return sentiment, confidence

In [9]:
# ----------------------
# User Interface
# ----------------------
def create_snt_analysis_interface(model_name):
    vocab = load_vocab(model_name)
    model = load_model(model_name)

    title = widgets.Label(value="Sentiment Analysis")
    text_input = widgets.Textarea(description="Sentence:", placeholder="e.g. I love this product!")
    output_area = widgets.Textarea(value="Result:", layout=widgets.Layout(height='50px'), disabled=True)
    tag_button = widgets.Button(description="Analyze")
    
    def on_infer_clicked(b):
        input_text = text_input.value
        if input_text.strip():
            sentiment, confidence = predict_sentiment(model, vocab, input_text, max_length=32)
            output_area.value = f"Sentiment: {sentiment.capitalize()}\nConfidence: {confidence*100:.2f}%"
        else:
            output_area.value = "Please enter some text."
    
    tag_button.on_click(on_infer_clicked)
    
    display(widgets.VBox([title, text_input, tag_button, output_area]))

In [10]:
create_snt_analysis_interface("text-cnn")

In [11]:
create_snt_analysis_interface("bi-rnn")

In [12]:
create_snt_analysis_interface("hybrid_cnn-rnn")